In [34]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [35]:
df = pd.read_csv("/home/condensed-sparsity/data/results/cifar10_sweep.csv")

In [42]:
slice = df.loc[(df["rigl.const_fan_in"] == True) & (df["rigl.dense_allocation"]==0.01) & (df.model == "resnet18")]
slice

,Name,Agent,State,Notes,Created,Runtime,Sweep,dataset,experiment.comment,experiment.name,...,wandb.project,wandb.start_method,paths.base,Learning Rate,acc,epoch,los,loss,new high score,test
141,resnet18_cifar10_sweep_0.01_True,awufzi7x,finished,-,2022-06-04T07:55:39.000Z,7720,4hsfzsa9,cifar10,sweep_0.01_True,resnet18_cifar10_sweep_0.01_True,...,condensed-rigl,thread,NaN,0.0008,NaN,250.0,NaN,0.269424,NaN,NaN
142,resnet18_cifar10_sweep_0.01_True,nahsbetr,finished,-,2022-06-04T05:47:42.000Z,26018,4hsfzsa9,cifar10,sweep_0.01_True,resnet18_cifar10_sweep_0.01_True,...,condensed-rigl,thread,NaN,0.0008,NaN,250.0,NaN,0.264209,NaN,NaN
143,resnet18_cifar10_sweep_0.01_True,awufzi7x,finished,-,2022-06-04T05:47:31.000Z,7685,4hsfzsa9,cifar10,sweep_0.01_True,resnet18_cifar10_sweep_0.01_True,...,condensed-rigl,thread,NaN,0.0008,NaN,250.0,NaN,0.270424,NaN,NaN


In [43]:
df["training.seed"].value_counts()

7303    31
2078    30
42      28
6037    28
8746    27
Name: training.seed, dtype: int64

In [33]:
slice = df.loc[(df["rigl.const_fan_in"] == True) & (df["rigl.dense_allocation"]==0.01)]
# slice = df.loc[df["rigl.dense_allocation"]==0.01]
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=slice.model, y=slice.accuracy)
)
fig